### NLP Lecture 2, comprehension exercises for ML + NL.

Matthew Stone   
Initial version, Spring 2018   


[Resampling for approximate permutation tests.][1]

[1]:https://en.wikipedia.org/wiki/Resampling_(statistics)

Rather than find an appropriate test statistic and interpret it according to a set procedure, it's often easier to simply simulate the results of your experiment assuming the null hypothesis (that is, assuming that any effects you observed were simply due to chance). You can use the fraction of time the simulation gets results similar to yours as an estimate of the  pp -value associated with your experimental results. This is technically known as an approximate permutation test in statistics.

Here, we assume that there's no difference in overall accuracy between the results of two classifiers. That means it doesn't really matter which classifier made which prediction; if we swap the classifiers randomly, we'll get similar results. Here sim_expt proposes to swap items from p1 and p2 at random and see whether the resulting simulated experiment passes the passed test. The test we'll be interested in is whether the difference in accuracy between the "two algorithms" exceeds the difference we observed in the actual experiment. 

In [3]:
from __future__ import print_function
import random
import numpy as np
import sklearn
import sklearn.metrics


In [4]:
def sim_expt(test, p1, p2):
    '''apply test to a simulated experiment 
       where the difference between p1 and p2 is randomly erased'''
    bits = [random.getrandbits(1) for i in range(0, len(p1))]
    return test([p1[i] if bits[i] else p2[i] for i in range(0,len(p1))],
                [p2[i] if bits[i] else p1[i] for i in range(0,len(p1))])

def eval_diff(data, p1, p2) :
    '''a test generator based on the performance difference between p1 and p2
       asks whether the result of a simulated experiment is at least as extreme
       as the observed accuracy difference between p1 and p2 on data'''
    diff = abs(sklearn.metrics.accuracy_score(data, p1) -
               sklearn.metrics.accuracy_score(data, p2))
    def test_diff(n, m) :
        return (abs(sklearn.metrics.accuracy_score(data, n) - 
                    sklearn.metrics.accuracy_score(data, m)) >= 
                diff)
    return test_diff

def mcmcp_diff(data, p1, p2, k) :
    '''the approximate permuatation test
       simulate an experiment like that used to obtain p1 and p2
       forgetting any difference between p1 and p2
       running the experiment k times
       and record the probability that the performance difference on data
       is as extreme as actually observed'''
    success = 0
    test = eval_diff(data, p1, p2)
    for i in range(0,k) :
        success += sim_expt(test, p1, p2)
    return success / float(k)

In [24]:
random.getrandbits(13)

2646

This exercise has two goals.
- Understand this code and appreciate the reasoning behind it.
- Get a feel for the kinds of results it gives: on the one hand, small data sets make it surprisingly difficult to detect even fairly substantial differences in performance; on the other hand, extremely large data sets can establish that even trivial differences are reliable.

### 1. Setup

Define `data` to contain 100K uniformly random bits.

Simulate an algorithm that classifies `data` with accuracy 80%, and store the results in `p1`

Simulate an algorithm that classifies `data` with accuracy 83%, and store the results in `p2`.

In [233]:

data = np.random.randint(2,size=100000)
p1 = []
p2 = []
for i in range(len(data)):
    if i in range(int(.8 *len(data))):
        a = data[i]
        p1.append(a)
    else:
        if data[i] == 0:
            p1.append(1)
        else:
            p1.append(0)

for j in range(len(data)):
    if j in range(int(.83 *len(data))):
        a = data[j]
        p2.append(a)
    else:
        if data[j] == 0:
            p2.append(1)
        else:
            p2.append(0)


In [235]:
p1 = np.array(p1)
p2 = np.array(p2)

In [238]:
def unison_shuffled_copies(a,b,c):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return list(a[p], b[p], c[p])

In [240]:
shuffled_values = unison_shuffled_copies(p1,p2,data)

In [243]:
p1 = shuffled_values[0]
p2 = shuffled_values[1]
data = shuffled_values[2]

In [246]:
sklearn.metrics.accuracy_score(p2,data)

0.83

In [278]:
sklearn.metrics.accuracy_score(p1,data)

0.8

### Variability, I

You can think of `data`, `p1`, and `p2` as
- 1 big experiment of size 100K: E1
- 10 experiments of size 10K: E10
- 100 experiments of size 1K: E100
- 1K experiments of size 100: E1000

It's easy to see that the accuracy of `p1` and `p2` in E1 has to be equal to the average accuracy across all the other experiments.  What is it?

It's more interesting to consider the standard deviations of the accuracy of `p1` and `p2` across experiments E10, E100, and E1000.  What are they?

In [247]:
E10_P1 = p1.reshape(10,10000)
E100_P1 = p1.reshape(100,1000)
E1000_P1 = p1.reshape(1000,100)

In [248]:
E10_P2 = p2.reshape(10,10000)
E100_P2 = p2.reshape(100,1000)
E1000_P2 = p2.reshape(1000,100)

In [249]:
E10_data = data.reshape(10,10000)
E100_data = data.reshape(100,1000)
E1000_data = data.reshape(1000,100)

In [260]:
def experiment_name(data,exp):
    acc_lst=[]
    for i in range(len(data)):
        count=0
        for j in range(len(data[i])):
            if data[i][j] == exp[i][j]:
                count +=1
        acc_lst.append(count/len(data[i]))
    return np.array(acc_lst)


In [263]:
W = experiment_name(E10_data,E10_P2)
np.mean(W)

0.8299999999999998

For P1 across all the experiemnts is ~80% and across P2 it is ~83%.

In [285]:
W = experiment_name(E1000_data,E1000_P1,E1000_P2)
W

array([0.81, 0.86, 0.85, 0.83, 0.84, 0.84, 0.83, 0.76, 0.76, 0.8 , 0.8 ,
       0.87, 0.77, 0.88, 0.81, 0.82, 0.83, 0.81, 0.82, 0.82, 0.85, 0.82,
       0.83, 0.87, 0.76, 0.86, 0.79, 0.85, 0.81, 0.89, 0.87, 0.79, 0.86,
       0.81, 0.85, 0.85, 0.87, 0.85, 0.85, 0.86, 0.77, 0.78, 0.81, 0.86,
       0.8 , 0.84, 0.82, 0.86, 0.71, 0.86, 0.76, 0.82, 0.85, 0.81, 0.79,
       0.84, 0.83, 0.81, 0.85, 0.81, 0.81, 0.8 , 0.85, 0.86, 0.94, 0.75,
       0.79, 0.84, 0.84, 0.8 , 0.89, 0.83, 0.82, 0.83, 0.83, 0.83, 0.84,
       0.83, 0.83, 0.91, 0.8 , 0.86, 0.83, 0.77, 0.79, 0.83, 0.83, 0.82,
       0.81, 0.83, 0.86, 0.79, 0.83, 0.81, 0.82, 0.86, 0.91, 0.84, 0.78,
       0.83, 0.81, 0.83, 0.78, 0.78, 0.81, 0.84, 0.87, 0.8 , 0.81, 0.85,
       0.79, 0.84, 0.91, 0.85, 0.87, 0.82, 0.82, 0.84, 0.78, 0.82, 0.82,
       0.83, 0.91, 0.91, 0.79, 0.89, 0.84, 0.79, 0.8 , 0.81, 0.74, 0.81,
       0.75, 0.9 , 0.76, 0.86, 0.88, 0.83, 0.81, 0.81, 0.81, 0.84, 0.78,
       0.8 , 0.8 , 0.87, 0.79, 0.85, 0.84, 0.83, 0.

In [283]:
print('E10',np.std(experiment_name(E10_data,E10_P2)))
print('E100',np.std(experiment_name(E100_data,E100_P2)))
print('E1000',np.std(experiment_name(E1000_data,E1000_P2)))

E10 0.004750578912090627
E100 0.012979984591670357
E1000 0.03804471053904865


The dispersion across the mean is higher when smaller number of observations are used for experiments. 


### Variability, II

Another way of asking a similar question: in how many of the experiments in E10, E100 and E1000, does p1 (misleadingly) wind up with better performance than p2?

In [297]:
def exp_variability(data,exp1,exp2):
    W = experiment_name(data,exp1)
    W1 = experiment_name(data,exp2)
    count=0
    assert len(W)==len(W1)
    for i in range(len(W)):
        if W1[i] > W[i]:
            count+=1
    return count,len(W)

In [298]:
exp_variability(E1000_data,E1000_P1,E1000_P2)

(967, 1000)

In [300]:
exp_variability(E100_data,E100_P1,E100_P2)

(100, 100)

In [302]:
exp_variability(E10_data,E10_P1,E10_P2)

(10, 10)

### Significance

Take a random experiment in each of E10, E100, and E1000.  What is the chance probability of a difference as large as what you see, according to the approximate permutation test, with k=2500?  (You are trading off accuracy versus waiting with this number.)

In [277]:
mcmcp_diff(E1000_data[0], E1000_P1[0], E1000_P2[0], 2500)

0.2464

### Conclusion

Larger experiments with smaller observations have higher value than a p-value of .05, so we fail to reject the null hypothesis and the accuracy we noticed were due to chance. But smaller experiments with larger observations does not bear such a high probability.